Name: Öykü Deniz Köse

I hereby declare that I observed the honour code of the university when preparing the homework.

## Pr?gr?mm?ng?H?m?w?rk

In this exercise we model a string of text using a Markov(1) model. For simplicity we only consider letters 'a-z'. Capital letters 'A-Z' are mapped to the corresponding ones. All remaining letters, symbols, numbers, including spaces, are denoted by '.'.


We have a probability table $T$ where $T_{i,j} = p(x_t = j | x_{t-1} = i)$  transition model of letters in English text for $t=1,2 \dots N$. Assume that the initial letter in a string is always a space denoted as $x_0 = \text{'.'}$. Such a model where the probability table is always the same is sometimes called a stationary model.

1. For a given $N$, write a program to sample random strings with letters $x_1, x_2, \dots, x_N$ from $p(x_{1:N}|x_0)$
1. Now suppose you are given strings with missing letters, where each missing letter is denoted by a question mark (or underscore, as below). Implement a method, that samples missing letters conditioned on observed ones, i.e., samples from $p(x_{-\alpha}|x_{\alpha})$ where $\alpha$ denotes indices of observed letters. For example, if the input is 't??.', we have $N=4$ and
$x_1 = \text{'t'}$ and $x_4 = \text{'.'}$, $\alpha=\{1,4\}$ and $-\alpha=\{2,3\}$. Your program may possibly generate the strings 'the.', 'twi.', 'tee.', etc. Hint: make sure to make use all data given and sample from the correct distribution. Implement the method and print the results for the test strings below. 
1. Describe a method for filling in the gaps by estimating the most likely letter for each position. Hint: you need to compute
$$
x_{-\alpha}^* = \arg\max_{x_{-\alpha}} p(x_{-\alpha}|x_{\alpha})
$$
Implement the method and print the results for the following test strings along with the log-probability  $\log p(x_{-\alpha}^*,x_{\alpha})$.
1. Discuss how you can improve the model to get better estimations.

In [15]:
test_strings = ['th__br__n.f_x.', '_u_st__n_.to_be._nsw_r__','i__at_._a_h_n_._e_r_i_g','q___t.___z._____t.__.___.__.']

Hint: The code below loads a table of transition probabilities for English text.

In [2]:
import csv
from IPython.display import display, Latex

alphabet = [chr(i+ord('a')) for i in range(26)]
alphabet.append('.')
letter2idx = {c:i for i,c in enumerate(alphabet)}

T = []
with open('transitions.csv') as csvfile:
    reader = csv.reader(csvfile, delimiter=',')
    for row in reader:
        T.append(row)

print('Example')
## p(x_t = 'u' | x_{t-1} = 'q')
display(Latex(r"$p(x_t = \text{'u'} | x_{t-1} = \text{'q'})$"))
print(T[letter2idx['q']][letter2idx['u']])
display(Latex(r"$p(x_t | x_{t-1} = \text{'a'})$"))
for c,p in zip(alphabet,T[letter2idx['a']]):
    print(c,p)


Example


<IPython.core.display.Latex object>

0.9949749


<IPython.core.display.Latex object>

a 0.0002835
b 0.0228302
c 0.0369041
d 0.0426290
e 0.0012216
f 0.0075739
g 0.0171385
h 0.0014659
i 0.0372661
j 0.0002353
k 0.0110124
l 0.0778259
m 0.0260757
n 0.2145354
o 0.0005459
p 0.0195213
q 0.0001749
r 0.1104770
s 0.0934290
t 0.1317960
u 0.0098029
v 0.0306574
w 0.0088799
x 0.0009562
y 0.0233701
z 0.0018701
. 0.0715219


In [3]:
import random
import numpy as np
from decimal import Decimal
#Part 1)

N=10 #Specify the length of the random string
choice='.'
random_string=""
random_string=random_string+choice
for j in range(N):
    cdf={}
    for i in range(len(alphabet)):
        cdf[alphabet[i]]=Decimal(str(T[letter2idx[choice]][letter2idx[alphabet[i]]]))
    pr= random.random()
    Fx = 0
    for c, p in cdf.items():
        Fx = Fx + p
        if (pr <= Fx):
            choice = c
            random_string=random_string+c
            break
            
print(random_string)

.wace..veas


In [97]:
#Part 3) I implemented Viterbi algorithm to find the most likely sequence

def find_subseq(previous,later,size):
    pron=np.multiply(np.ones((int(len(alphabet)),int(size))),-5)
    if(previous=='!'):
        for i in range(int(len(alphabet))):
            pron[i,0]=np.log(1/len(alphabet))
            
    else:
        for i in range(int(len(alphabet))):
            if(float(T[letter2idx[previous]][letter2idx[alphabet[i]]])==float(0)):
                pron[i,0]=-10
            else:
                pron[i,0]=np.log(float(T[letter2idx[previous]][letter2idx[alphabet[i]]]))
               
    ind=np.multiply(np.ones(size),-1)
    for i in range(1,int(size)):
        for k in range(int(len(alphabet))):
            if(float(T[letter2idx[alphabet[0]]][letter2idx[alphabet[k]]])==float(0)):
                pron[k,i]=-10
            else:
                pron[k,i]=pron[0,i-1]+np.log(float(T[letter2idx[alphabet[0]]][letter2idx[alphabet[k]]]))
               
            ind[i-1]=0
            for j in range(1,int(len(alphabet))):
                if(float(T[letter2idx[alphabet[j]]][letter2idx[alphabet[k]]])==float(0)):
                    pron[k,i]=pron[k,i]
                elif(pron[j,i-1]+np.log(float(T[letter2idx[alphabet[j]]][letter2idx[alphabet[k]]]))>pron[k,i]):
                    
                    pron[k,i]=pron[j,i-1]+np.log(float(T[letter2idx[alphabet[j]]][letter2idx[alphabet[k]]]))
                    ind[i-1]=j
    if(later=='!'):
        ind[size-1]=np.argmax(pron[:,size-1])
        log_prob=max(pron[:,size-1])
    else:
        ind[size-1]=0
        if(float(T[letter2idx[alphabet[0]]][letter2idx[later]])==float(0)):
            prob=-100
        else:
            prob=pron[0,size-1]+np.log(float(T[letter2idx[alphabet[0]]][letter2idx[later]]))
        log_prob=prob
        for k in range(1,int(len(alphabet))):
            if(float(T[letter2idx[alphabet[k]]][letter2idx[later]])==float(0)):
                prob=prob
            elif(pron[k,size-1]+np.log(float(T[letter2idx[alphabet[k]]][letter2idx[later]]))>prob):
                prob=pron[k,size-1]+np.log(float(T[letter2idx[alphabet[k]]][letter2idx[later]]))
                ind[size-1]=k
                log_prob=prob
            
    return ind,log_prob
                
def prepare_test(test):
    un=0
    test_dict={}
    for i in range(len(test)):
        if(test[i]=='_' and i==0):
            test_dict['init'+str(un)]='!'
            test_dict['ind'+str(un)]=0
            un=un+1
            test_dict['len'+str(un-1)]=1
        elif(test[i]=='_' and test[i-1]!='_'):
            test_dict['init'+str(un)]=test[i-1]
            test_dict['len'+str(un)]=1
            test_dict['ind'+str(un)]=i
            un=un+1
        elif(test[i]=='_' and test[i-1]=='_'):
            test_dict['len'+str(un-1)]=test_dict['len'+str(un-1)]+1
        if(test[i]=='_' and i==len(test)-1):
            test_dict['fin'+str(un-1)]='!'
        elif(test[i]=='_' and test[i+1]!='_' and i!=len(test)-1):
            test_dict['fin'+str(un-1)]=test[i+1]
    test_m=""
    
    for i in range(un):
        ind,log_prob=find_subseq(test_dict['init'+str(i)],test_dict['fin'+str(i)],test_dict['len'+str(i)])
        print("For the "+str(i+1)+"th missing letter/letters the maximizing letter series has the log probability"+str(log_prob))
        if(i==0):
            test_m=test_m+test[0:test_dict['ind'+str(i)]]
        else:
            test_m=test_m+test[len(test_m):test_dict['ind'+str(i)]]
        for j in range(len(ind)):
            test_m=test_m+alphabet[int(ind[j])]
    test_m=test_m+test[len(test_m):]
            
            
        
    return test_m
    
        
for i in range(len(test_strings)):
    print(prepare_test(test_strings[i]))        
        
        

            
            
            
 
    

            
        
    

For the 1th missing letter/letters the maximizing letter series has the log probability-4.933091002188418
For the 2th missing letter/letters the maximizing letter series has the log probability-5.275963684161198
For the 3th missing letter/letters the maximizing letter series has the log probability-6.93208128464704
the.brean.fex.
For the 1th missing letter/letters the maximizing letter series has the log probability-3.300874634276711
For the 2th missing letter/letters the maximizing letter series has the log probability-4.896730029412735
For the 3th missing letter/letters the maximizing letter series has the log probability-4.376744460254719
For the 4th missing letter/letters the maximizing letter series has the log probability-2.21088999523798
For the 5th missing letter/letters the maximizing letter series has the log probability-5.182512738347487
For the 6th missing letter/letters the maximizing letter series has the log probability-3.7813002863190257
For the 7th missing letter/lette

In [ ]:
# Part 4)

# The order of the Markov model can be increased so that better results can be achieved with increasing computational cost.